In [1]:
# DATA_PATH = 'data/hdmb'
DATA_PATH = ['data',]
### csv must be  id|xxx|xxx|nose|LEye.......


alphaI2W = [ "nose","LEye","REye","LEar","REar","LShoulder","RShoulder", "LElbow","RElbow",\
"LWrist", "RWrist","LHip","RHip", "LKnee","Rknee", "LAnkle","RAnkle"]

# tagI2W = ["jump","run","sit","stand","walk"] # 5
tagI2W = ["FallingDown","Standing"] # 9
tagW2I = {w:i for i,w in enumerate(tagI2W)}

In [2]:
import os
import pandas as pd 
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

parpath = os.path.dirname(os.getcwd())
sys.path.append(parpath)

torch.manual_seed(42)

In [3]:
N_EPOCHS = 1000
BATCH_SIZE = 32
SHUFFLE = True
INPUT_DIM = 34
CLASS_NUM = len(tagI2W)
# device = 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from actRec.models import FcLstm
model = FcLstm(INPUT_DIM,CLASS_NUM,initrange=1.5).to(device)
criterion = nn.CrossEntropyLoss().to(device)
# model.double()
# model.cuda()
model

FcLstm(
  (fc1): Linear(in_features=34, out_features=64, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (lstm): LSTM(64, 64, batch_first=True)
  (fc3): Linear(in_features=64, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)

In [5]:
from dataset import SeqDataset
from actRec.F import normalize_min_
class PoseActDataFromCsvDir():
    def __init__(self, csvdirpaths,use_large=True):
        data  = []
        label = []
        for csv_dir_path in csvdirpaths:
            csvlist = os.listdir(csv_dir_path)
            for fname in csvlist:
                if(os.path.isdir(fname)):continue
                if(os.path.splitext(fname)[1] != '.csv'):continue
                    
                csvf = os.path.join(csv_dir_path,fname)
                df = pd.read_csv(csvf)
                features = df.iloc[:,4:].values
                pose_class = df['pos_class'].values
                normalize_min_(features)
                
                for feats, lbs in zip(features, pose_class):
                    data.append(feats)
                    label.append(tagW2I[lbs])

        self.dataset = np.array(data)
        self.labels  = np.array(label)

    def __len__(self):
        return len(self.dataset)
    
    def split(self,test_size=0.2, random_state=42):
        X_train, X_test, Y_train, Y_test = train_test_split(self.dataset, self.labels, test_size=test_size, random_state=random_state)
        return SeqDataset(X_train,Y_train),SeqDataset(X_test,Y_test)

In [6]:
# from torch_inspect import summary
# summary(model,[(1,7,34),(1)])
PoseActDataFromCsvDir(DATA_PATH,True).__len__()

18189

In [7]:
from sklearn.model_selection import train_test_split
train_data,test_data = PoseActDataFromCsvDir(DATA_PATH,True).split()

In [8]:
len(train_data), len(test_data)

(14551, 3638)

In [9]:
train_data[0]

{'data': tensor([0.9643, 0.4085, 0.9643, 0.3944, 0.9286, 0.3944, 0.8929, 0.1690, 0.7321,
         0.3521, 0.8571, 0.0845, 0.5893, 0.4085, 0.9107, 0.0000, 0.8750, 0.6761,
         1.0000, 0.4225, 0.9821, 0.5634, 0.5893, 0.5070, 0.3750, 0.5634, 0.7500,
         0.8310, 0.4643, 1.0000, 0.2500, 0.6620, 0.0000, 0.8310]),
 'label': 0}

In [10]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,collate_fn=SeqDataset.collate_fn_pad,shuffle=SHUFFLE)
test_loader  = DataLoader(test_data, batch_size=BATCH_SIZE,collate_fn=SeqDataset.collate_fn_pad,shuffle=SHUFFLE)

In [11]:
def model_eval(batch):
    label_ = torch.LongTensor(batch['label']).to(device)
    input_= batch['data'][0].to(device)
    len_ = batch['data'][1].to(device)
    print(len(len_),len(input_))
    
    hc_ = (torch.randn(1,BATCH_SIZE,32).to(device),
          torch.randn(1,BATCH_SIZE,32).to(device))#LSTM out
    print(hc_[0].shape, hc_[1].shape)
    out,_hc = model(input_,len_,hc_) # data,datalen
    out = out.view(-1,9)
    loss = criterion(out, label_)
    
    batch_acc = (out.argmax(1) == label_).sum().item()
    return loss,batch_acc

In [12]:
# optimizer = optim.SGD(model.parameters(), lr=0.05)
optimizer = optim.Adadelta(model.parameters())
scheduler = optim.lr_scheduler.StepLR(optimizer, 100, gamma=0.4)

In [13]:
import procession as pc
import time
_epoch = -1

t = time.localtime(time.time())
timestr = '%d-%d-%d_%d-%d'%(t[0],t[1],t[2],t[3],t[4])
savepath = 'model/act_fcLstm%d_'%len(tagI2W)
savepath +=timestr

os.chdir(parpath)
if not os.path.exists(savepath):
    os.makedirs(savepath)

In [14]:
import hiddenlayer as hl
train_his = hl.History()
vaild_his = hl.History()
canvas1 = hl.Canvas()
for epoch in range(_epoch+1,N_EPOCHS+1):
    start_time = time.time()
    train_loss, train_acc = pc.train_epoch(model,train_loader,model_eval,optimizer,scheduler,BATCH_SIZE)
    valid_loss, valid_acc = pc.evaluate(model,test_loader,model_eval,BATCH_SIZE)
    train_his.log(epoch, loss=train_loss, accuracy=train_acc)
    vaild_his.log(epoch, loss=valid_loss, accuracy=valid_acc)
    with canvas1:
        canvas1.draw_plot([train_his["loss"], vaild_his["loss"]],labels=["train", "vaild"])
        canvas1.draw_plot([train_his["accuracy"], vaild_his["accuracy"]],labels=["train", "vaild"])
    pc.print_train_info(start_time,epoch,train_loss,train_acc,valid_loss,valid_acc)
    _epoch=epoch
    info =  {'train_loss':train_loss,'train_acc':train_acc,'valid_loss':valid_loss,'valid_acc':valid_acc}
    if epoch % 500 ==0 and epoch != 0: 
        pc.save_model_with_info(model,optimizer,tagI2W,info,epoch,savepath)

2020-12-25 20:45:17.998 | ERROR    | __main__:<module>:7 - An error has been caught in function '<module>', process 'MainProcess' (80182), thread 'MainThread' (140591315527488):
Traceback (most recent call last):

  File "/home/rasho/anaconda3/envs/alphapose/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
                └ ModuleSpec(name='ipykernel_launcher', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fddf6f29c18>, origin='...
  File "/home/rasho/anaconda3/envs/alphapose/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
         └ <code object <module> at 0x7fddf6f93db0, file "/home/rasho/anaconda3/envs/alphapose/lib/python3.6/site-packages/ipykernel_lau...
  File "/home/rasho/anaconda3/envs/alphapose/lib/python3.6/site-packages/ipykernel_launcher.py", line 16

32 32
torch.Size([1, 32, 32]) torch.Size([1, 32, 32])
fc1 torch.Size([32, 64])


TypeError: 'NoneType' object is not iterable